In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:

data=pd.read_csv('spam_clean.csv',encoding='latin-1')
data.head()

,type,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


# cleaning messages

In [16]:
import nltk 
import re
nltk.download('punkt_tab')
nltk.download('stopwords')
from nltk import word_tokenize,sent_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\apattan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\apattan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
def clean_message(sentence):
  words=nltk.word_tokenize(sentence)
  cleaned_msg=""
  for word in words:
    word=word.lower()
    word=re.sub(r'[^\w\s]','',word)
    if word!="" and word not in stopwords.words('english'):
      cleaned_msg=cleaned_msg+" "+word
  return cleaned_msg.strip()
  
    

In [23]:
data['cleaned_msg']=data['message'].apply(clean_message)
data.head()

,type,message,cleaned_msg
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah nt think goes usf lives around though


# data preparation for naive bayes

In [ ]:
class NaiveBayes:
    def __init__(self):
        self.common_words=[]

    
    def fit(self,X,y):
        self.